In [44]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, cohen_kappa_score, precision_score, recall_score, \
    precision_recall_curve

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import clone
from tqdm import tqdm

import os
from gensim.models import KeyedVectors
from gensim.downloader import base_dir

import random

In [82]:
#Dataframe  generated in analye-data.ipynb
df = pd.read_json("./Trec_data/Features_Labeled.json", orient='records',lines=True)
df

,eventID,eventType,postID,postCategories,postPriority,text,entities,extended_entities,favorite_count,hashtagEntities,...,bool_EmergingThreats,bool_Weather,bool_Volunteer,bool_Hashtags,bool_OriginalEvent,bool_ContextualInformation,bool_InformationWanted,bool_GoodsServices,bool_SearchAndRescue,bool_Location
0,fireColorado2012,wildfire,212311994286620672,[MultimediaShare],Unknown,RT @CBSDenver: The copter is on the way to the...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,False,False,False,False,False,False,False,False,False,False
1,fireColorado2012,wildfire,217746356842926080,[MultimediaShare],Medium,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,False,False,False,False,False,False,False,False,False,False
2,fireColorado2012,wildfire,217732012314861568,[FirstPartyObservation],Medium,2 wildfires in Boulder County. We can see smok...,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",None,0.0,None,...,False,False,False,False,False,False,False,False,False,False
3,fireColorado2012,wildfire,216961334129078272,[Discussion],Low,RT @Jon_G3: Seeing 1/3 of Colorado on fire mak...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,False,False,False,False,False,False,False,False,False,False
4,fireColorado2012,wildfire,212552860590813184,[MultimediaShare],Medium,RT @dhorning11: RT @LarimerCounty: #HighParkFi...,"{'symbols': [], 'urls': [{'expanded_url': 'htt...",None,0.0,None,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38071,tennesseeTornadoOutbreak2020,tornado,1235236359310368768,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Hottest Google Search in 31.2 hrs. Nashville t...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,False,True,False,False,False,True,False,False,False,True
38072,tennesseeTornadoOutbreak2020,tornado,1235337290144239616,"[ThirdPartyObservation, Location, MultimediaSh...",Low,A live report is next on the Nashville tornado...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,False,False,False,False,False,False,False,False,False,True
38073,tennesseeTornadoOutbreak2020,tornado,1235258820139638784,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Officials are still cleaning up after tornadoe...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,False,True,False,False,False,False,False,False,False,True
38074,tennesseeTornadoOutbreak2020,tornado,1235253249957126144,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Putnam County: Cookeville area tornado victims...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,False,True,False,False,False,False,False,False,False,True


# **`Generate Heldout Events`**

In [84]:
fullEventTypes = df['eventType'].unique()
eventTypes = []
for event in fullEventTypes:
    events = df.loc[df['eventType'] == event]['eventID'].unique()
    if events.size > 1:
        eventTypes.append(event)
print(eventTypes)

['wildfire', 'earthquake', 'flood', 'typhoon', 'shooting', 'bombing', 'storm']


In [142]:
heldout_events = {}

#Choose heldout event and saves in the heldout_events dataframe
for event in eventTypes:
    crises = df.loc[df['eventType']==event]['eventID'].unique()
    heldout_events[event]=[crises[random.choice(np.arange(crises.size))]]

heldout_events = pd.DataFrame.from_dict(heldout_events)
heldout_events.to_json('./Trec_data/heldout_events.json')

(5,)
(11,)
(11,)
(9,)
(10,)
(3,)
(2,)


In [86]:
#Simple read to keep the index by events
heldout_events = pd.read_json('./Trec_data/heldout_events.json')
heldout_events

,wildfire,earthquake,flood,typhoon,shooting,bombing,storm
0,fireYMM2016,guatemalaEarthquake2012,whaleyBridgeCollapse2020,cycloneKenneth2019,brooklynBlockPartyShooting2020,westTexasExplosion2013,southeastTornadoOutbreak2020


# **`Model Related Methods`**

In [87]:
def train_data(data, column, heldout_ids):
    training = data.loc[~data[column].isin(heldout_ids)]
    
    return training

def test_data(data, column, heldout_ids):
    test = data.loc[data[column].isin(heldout_ids)]
    
    return test

In [88]:
def generate_model(data, features, target, modelType):
    
    model = clone(modelType)
    model.fit(data[features], data[target])
    
    return model

In [89]:
def generate_model_by_events(data, features, target, modelType):
    modelList = {}
    for event in tqdm(eventTypes):
        #Create training and test dataframe
        eventDF = df.loc[df['eventType']==event]
        
        training = train_data(eventDF, 'eventID', heldout_events[event])
        
        #generate event specific model
        model = generate_model(training, features, target, modelType)
        
        #Add model to list
        modelList[event] = model
        
        #print('')
    return modelList

# **`Generate Generic Variables`**

In [124]:
features = ["num_chars", "num_chars_total", 
            "num_terms", "num_words", "num_unique_words", "vader neg", "vader pos",
            "vader neu", "vader compound", 
            "num_hashtags", "num_mentions", 
            "num_urls", 
            "is_retweet", "num_media",
            "is_verified", 
            "caps_ratio"]

#I think you need to make a list of lists

rf_params = {
    'random_state': 1337,
    'class_weight': 'balanced',
    'n_estimators': 128, 
    'n_jobs': -1,
    'max_depth': 50,
    'max_features': 14,
    'min_samples_leaf': 33,
    'min_samples_split': 96,
    'verbose': 1
}

heldout_event_ids = []

for item in heldout_events.loc[0]:
    heldout_event_ids.append(item)
    
heldout_event_ids

#Training data withholding all heldout events for general models
generalTraining = train_data(df, 'eventID', heldout_event_ids) #Check this
print(generalTraining.shape)

(33438, 70)


# **`Generate postPriority Models`**

In [92]:
prioLabel = 'postPriority'
prioModel = RandomForestClassifier(**rf_params) #(**modelParameters)

#generate general model
genPrioModel = generate_model(generalTraining, features, prioLabel, prioModel)

#generate event specific models
specPrioModels = generate_model_by_events(df, features, prioLabel, prioModel)

100%|██████████| 7/7 [00:10<00:00,  1.48s/it]


# **`Generate postCategories Models`**

In [236]:
#model troubleshooting

'''ValueError: You appear to be using a legacy multi-label data representation. 
Sequence of sequences are no longer supported; use a binary array or sparse matrix 
instead - the MultiLabelBinarizer transformer can convert to this format.'''

from sklearn.preprocessing import MultiLabelBinarizer

def generate_cat_model(data, features, target, modelType):
    
    #print(MultiLabelBinarizer().fit_transform(data[target]).shape) this has 25 in second dimension
    
    model = clone(modelType)
    model.fit(data[features], MultiLabelBinarizer().fit_transform(data[target]))
    
    return model

def generate_cat_model_by_events(data, features, target, modelType):
    modelList = {}
    for event in tqdm(eventTypes):
        #Create training and test dataframe
        eventDF = df.loc[df['eventType']==event]
        
        training = train_data(eventDF, 'eventID', heldout_events[event])
        
        #generate event specific model
        model = generate_cat_model(training, features, target, modelType)
        
        #Add model to list
        modelList[event] = model
        
        #print('')
    return modelList

In [125]:
catModel = MultiOutputClassifier(RandomForestClassifier(**rf_params))

#generate general model
catLabel = 'postCategories'
genCatModel =  generate_cat_model(generalTraining, features, catLabel, catModel)

#generate event specific models
specCatModels = generate_cat_model_by_events(df, features, catLabel, catModel)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:   11.1s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:    8.8s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:   11.1s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:   11.9s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:    9.4s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:   11.7s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [128]:
def save_model(model, filename):
    pickle.dump(model, open(filename, 'wb'))
    
def load_model(filename):
    model = pickle.load(open(filename, 'rb'))
    return model

In [130]:
import pickle

#Save postPriority models
baseFilename = 'PrioModel.pkl'
save_model(genPrioModel, 'gen' + baseFilename)

for event in eventTypes:
    save_model(specPrioModels[event], event + baseFilename)

In [131]:
#Save postCategories models
baseFilename = 'CatModel.pkl'
save_model(genCatModel, 'gen' + baseFilename)

for event in eventTypes:
    save_model(specCatModels[event], event + baseFilename)

# **`Test All Models`**

In [132]:
#Load All Models
genPrioModel
specPrioModels
genCatModel
specCatModels

{'wildfire': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight='balanced',
                                                        criterion='gini',
                                                        max_depth=50,
                                                        max_features=14,
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=33,
                                                        min_samples_split=96,
                                                        min_weight_fraction_l

In [133]:
def test_prio_model(data, features, target, model):
    X_test = data[features]
    y_test = data[target]
    y_infer_local = model.predict(X_test)
    local_f1 = f1_score(y_test, y_infer_local, average="weighted")
    local_score = model.score(X_test, y_test)
    
    #print("\tAccuracy:", local_score)
    #print("\tF1:", local_f1)
    return [local_score, local_f1]

In [163]:
specPrioModels['wildfire']

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=50, max_features=14,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=33, min_samples_split=96,
                       min_weight_fraction_leaf=0.0, n_estimators=128,
                       n_jobs=-1, oob_score=False, random_state=1337, verbose=0,
                       warm_start=False)

In [164]:
#Test postPriority models
prioDF = pd.DataFrame(columns=['Event', 'genAccuracy', 'genF1', 'specAccuracy', 'specF1'])
for event in eventTypes:
    eventDf = df.loc[df['eventType']==event]
    testDf = test_data(eventDf, 'eventID', heldout_events[event])
    genScores = test_prio_model(testDf, features, prioLabel, genPrioModel)
    specScores = test_prio_model(testDf, features, prioLabel, specPrioModels[event])
    prioDF = prioDF.append({'Event':event, 'genAccuracy':genScores[0], 'genF1':genScores[1],
               'specAccuracy':specScores[0], 'specF1':specScores[1]}, ignore_index=True)
prioDF

,Event,genAccuracy,genF1,specAccuracy,specF1
0,wildfire,0.483516,0.602780,0.681319,0.771502
1,earthquake,0.631143,0.647710,0.612789,0.629350
2,flood,0.705346,0.707400,0.653194,0.663981
3,typhoon,0.551370,0.647155,0.801370,0.839506
4,shooting,0.736842,0.750320,0.659774,0.680136
5,bombing,0.429167,0.514174,0.645139,0.710842
6,storm,0.755556,0.755577,0.453333,0.363538


In [226]:
import sklearn.preprocessing

def test_cat_model(data, features, target, model):
    X_test = data[features]
    y_test = MultiLabelBinarizer().fit_transform(data[target]) # labels don't match dimension because each event doesn't have all categories
    y_infer_local = model.predict(X_test) #25 categories, 25 in the second dimension

    local_f1 = f1_score(y_test, y_infer_local, average="weighted")
    local_score = model.score(X_test, y_test)
    
    #print("\tAccuracy:", local_score)
    #print("\tF1:", local_f1)
    return [local_score, local_f1]

In [340]:
def test_cat_model(data, features, target, model):
    X_test = data[features]
    y_test = MultiLabelBinarizer().fit_transform(data[target]) # 11 in second dimension
    print("shape:", y_test.shape)

    y_infer_local = model.predict(X_test) #25 categories, 25 in the second dimension
    print("target shape:", y_infer_local.shape)

    local_f1 = f1_score(y_test, y_infer_local, average="weighted")
    local_score = model.score(X_test, y_test)
    
    #print("\tAccuracy:", local_score)
    #print("\tF1:", local_f1)
    return [local_score, local_f1]

In [381]:
#generating categories list

def category_list(eventDf):
    #this should return a list
    categories = []
    for i in eventDf['postCategories']:
        for j in i:
            categories.append(j)
            
    categories = (set(categories))
    return list(categories)

#this one works for general
    
''' the labels for y_test and y_infer_local do not match because not all info_types are represented 
    in each event_type. E.g. for wildfire, only 11 info_types are represented, so the y_test matrix 
    has a second dimension of 11, not 25.
    
    Solution would probably be to "pad" the info_types to make sure all are
    represented'''
    
def test_gen_cat_model(data, features, target, model):
    categories = category_list(eventDf)
    add_row = data.append(pd.Series(name='delete')) #adds an empty row
    add_row[target] = add_row[target].astype('object') #enables lists to be added to cell
    add_row.at['delete', target] = categories #adds list of all categories to cell to pad
    
    y_test = MultiLabelBinarizer().fit_transform(add_row[target]) #this should yield 25 in second dimension
    print("shape:", y_test.shape)
    
    y_test = y_test[:-1, :]
    print("shape after drop:", y_test.shape)

    X_test = data[features]
    y_infer_local = model.predict(X_test)
    print("target shape:", y_infer_local.shape)
    
    local_f1 = f1_score(y_test, y_infer_local, average="weighted")
    local_score = model.score(X_test, y_test)
    
    #print("\tAccuracy:", local_score)
    #print("\tF1:", local_f1)
    return [local_score, local_f1]

In [392]:
#specific
    
''' Similar solution, but the target shape varies depending on event'''
    
def test_cat_model(data, features, target, model):
    print('EVENT:', event)
    categories = category_list(eventDf)
    add_row = data.append(pd.Series(name='delete')) #adds an empty row
    add_row[target] = add_row[target].astype('object') #enables lists to be added to cell
    add_row.at['delete', target] = categories #adds list of all categories to cell to pad
    
    y_test = MultiLabelBinarizer().fit_transform(add_row[target]) #this should yield 25 in second dimension
    print("shape:", y_test.shape)
    
    y_test = y_test[:-1, :]
    print("shape after drop:", y_test.shape)

    X_test = data[features]
    y_infer_local = model.predict(X_test)
    print("target shape:", y_infer_local.shape)
    
    local_f1 = f1_score(y_test, y_infer_local, average="weighted")
    local_score = model.score(X_test, y_test)
    
    #print("\tAccuracy:", local_score)
    #print("\tF1:", local_f1)
    return [local_score, local_f1]

In [386]:
#Test postCategories models
catDF = pd.DataFrame(columns=['Event', 'genAccuracy', 'genF1', 'specAccuracy', 'specF1'])
for event in eventTypes:
    eventDf = df.loc[df['eventType']==event]
    testDf = test_data(eventDf, 'eventID', heldout_events[event])
    genScores = test_gen_cat_model(testDf, features, catLabel, genCatModel)
    specScores = test_cat_model(testDf, features, catLabel, specCatModels[event])
    catDF = catDF.append({'Event':event, 'genAccuracy':genScores[0], 'genF1':genScores[1],
               'specAccuracy':specScores[0], 'specF1':specScores[1]}, ignore_index=True)
catDF

shape: (547, 25)
shape after drop: (546, 25)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

target shape: (546, 25)


[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Us

EVENT: wildfire
shape: (547, 25)
shape after drop: (546, 25)


[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Us

target shape: (546, 24)


[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    0.0s finished


ValueError: Multi-label binary indicator input with different numbers of labels

# Troubleshooting (DO NOT DELETE)

In [391]:
#troubleshooting category dimensions

eventDf = df.loc[df['eventType']=='wildfire']

def category_list(eventDf):
    #this should return a list
    categories = []
    for i in eventDf['postCategories']:
        for j in i:
            categories.append(j)
            
    categories = (set(categories))
    return list(categories)

categories = category_list(eventDf)
len(categories)

25

In [352]:
for event in eventTypes:
    print(event)

wildfire
earthquake
flood
typhoon
shooting
bombing
storm


In [282]:
#troubleshooting

eventDf = df.loc[df['eventType']=='wildfire']
testDf = test_data(eventDf, 'eventID', heldout_events['wildfire'])


x = [i for i in testDf['postCategories']]

#messing around

d = {'col1': [1, 2], 'col2': [3, 4]}

dd = pd.DataFrame(d)

new = dd.append(pd.Series(name='delete'))

new['col1'] = new['col1'].astype('object')

new.at['delete', 'col1'] = categories

new.drop(['delete'])

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li